In [34]:
import pandas as pd
import json
import urllib
import numpy as np
from sklearn import metrics

import tensorflow as tf
import numpy as np
import os
import sys
import random
from PIL import Image
from PIL import ImageEnhance
import PIL.ImageOps
from six.moves import cPickle as pickle
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Conv3D, MaxPooling3D, LSTM, GlobalAveragePooling2D
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3

# Image Data Processing

In [20]:
df=pd.read_csv('/home/ubuntu/data/whole_data.csv')

/home/ubuntu/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,4,12,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
df=df[df['tmdb_id'].isnull()==False]

In [22]:
len(df['poster_path'])

63823

In [ ]:
# download img
i = 26500
for i in range(len(df)):
    urllib.urlretrieve('https://image.tmdb.org/t/p/w154'+df.poster_path[i],'/home/ubuntu/poster/'+str(df.tmdb_id[i])+'.jpg')

In [7]:
#load label
df_y = pd.read_csv('/home/ubuntu/data/whole_data_yonly.csv')
df_y=df_y.ix[:,'tmdb_id':'recoded_y']

In [8]:
IMAGE_WIDTH=154
IMAGE_HEIGHT=154
NUM_CHANNELS=3
PIXEL_DEPTH=255.0
DATA_PATH = '/home/ubuntu/poster/'
poster=[0 for i in range(26000)]

In [9]:
tmdb_id=[0 for i in range(26000)]

In [10]:
def to_rgb(im): 
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, :] = im[:, :, np.newaxis]
    return ret
def read_image_from_file(file_path):
    img = Image.open(file_path).convert('RGB')
      #downsample image
    img = img.resize((154,154), Image.ANTIALIAS)
    pixel_values = np.array(img.getdata())
    return np.reshape(pixel_values, [IMAGE_WIDTH,IMAGE_HEIGHT, NUM_CHANNELS])
def scale_pixel_values(dataset):
    return (dataset - 255.0 / 2.0) / 255.0

In [11]:
i=0
j=0
for filename in os.listdir(DATA_PATH):
    if i <26000:
        idx= int(filename.split('.')[0])
        image=read_image_from_file(DATA_PATH+filename)
        poster[i]=image
        tmdb_id[j]=idx
        i+=1
        j+=1

In [12]:
df_img = pd.DataFrame({'img':poster, 'tmdb_id':tmdb_id})

In [13]:
l=['tmdb_id','cluster_1','cluster_2','cluster_3','cluster_4','cluster_5','cluster_6','cluster_7']
df_3 = df.ix[:,l]
df_3.head()

,tmdb_id,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
0,100,0,0,0,1,0,1,0
1,10001,0,0,1,1,0,0,0
2,10002,1,0,0,1,0,1,0
3,10003,1,1,0,0,1,0,1
4,10004,0,1,0,1,1,1,0


In [14]:
#df_whole=pd.merge(df_img, df_y,on = 'tmdb_id', how='inner')
df_whole=pd.merge(df_img, df_3,on = 'tmdb_id', how='inner')

In [15]:
df_whole.head()

,img,tmdb_id,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
0,"[[[253, 249, 248], [253, 249, 248], [253, 249,...",43819,1,0,0,1,0,0,0
1,"[[[19, 12, 23], [23, 16, 26], [26, 21, 30], [2...",36996,0,1,0,1,0,0,1
2,"[[[182, 153, 145], [182, 153, 145], [179, 151,...",16520,1,0,1,1,0,0,0
3,"[[[242, 244, 243], [245, 247, 245], [242, 245,...",29960,0,1,0,0,0,0,0
4,"[[[50, 51, 43], [21, 22, 14], [22, 23, 15], [1...",41678,0,0,0,1,0,0,0


In [ ]:
df_whole.to_pickle('whole_img.p')

In [4]:
df_whole= pd.read_pickle('whole_img.p')
df_whole.head()

,img,tmdb_id,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
0,"[[[253, 249, 248], [253, 249, 248], [253, 249,...",43819,1,0,0,1,0,0,0
1,"[[[19, 12, 23], [23, 16, 26], [26, 21, 30], [2...",36996,0,1,0,1,0,0,1
2,"[[[182, 153, 145], [182, 153, 145], [179, 151,...",16520,1,0,1,1,0,0,0
3,"[[[242, 244, 243], [245, 247, 245], [242, 245,...",29960,0,1,0,0,0,0,0
4,"[[[50, 51, 43], [21, 22, 14], [22, 23, 15], [1...",41678,0,0,0,1,0,0,0


In [5]:
train_X=np.zeros((10000,154,154, 3))

In [6]:
x= df_whole.img[:10000].values

In [7]:
for i in range(10000):
    train_X[i]=x[i]

In [8]:
train_Y=df_whole.ix[:9999,2: ].values

In [9]:
vali_X=np.zeros((5000,154,154, 3))
x= df_whole.img[10000:15000].values
for i in range(5000):
    vali_X[i]=x[i]
vali_Y=df_whole.ix[10000:14999,2: ].values

In [10]:
x_test= np.zeros((5000,154,154, 3))
x= df_whole.img[15000:20000].values
for i in range(5000):
    x_test[i]=x[i]
y_test=df_whole.ix[15000:19999,2: ].values

# Model from scratch
# Training Process

# 1. Try different CNN structure
## Structure 1

As for the model from scratch, we started our idea by considering build convolutional neural network. The reason that we choose CNN is because the raw data we have is the image data. We began our experiment with one input layer(conv2D) and one con2D hidden layer, one maxPooling layer and one output layer with 7 output units(sigmoid) with dropOut rate 0.25. 

The input data is the image pixel data matrices. Each image has 154x154 dimensions and 3 channels. The response variable is n x 7. Each image will have seven response variables. The seven columns are the cluster results we generated in the previous milestone.


In [66]:
input_shape = ( 154,154, 3)

In [13]:
batch_size= 500
epochs=10

In [20]:
model1 = Sequential()
model1.add(Conv2D(32, (5, 5), activation='relu', input_shape=input_shape))
model1.add(Conv2D(32, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))


model1.add(Flatten())
model1.add(Dense(7,activation='sigmoid'))

# prints out a summary of the model architecture
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 150, 150, 32)      2432      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 175232)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 1226631   
Total params: 1,238,311.0
Trainable params: 1,238,311.0
Non-trainable params: 0.0
____________________________________________________________

In [21]:
sgd = SGD(lr=0.1, momentum=0.9)
model1.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
history = model1.fit(train_X, train_Y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(vali_X, vali_Y))

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 38s - loss: 3.7977 - acc: 0.7582 - val_loss: 3.6228 - val_acc: 0.7749

In [22]:
score = model1.evaluate(x_test, y_test, verbose=0)
score[1]

0.76611429643630979

In [24]:
# F1 score
pred_val_Y= model1.predict(x_test)
print("Precision CNN(from scratch)", round(metrics.precision_score(y_test, pred_val_Y, average='weighted'), 3))
print("Recall CNN(from scratch)", round(metrics.recall_score(y_test, pred_val_Y, average='weighted'), 3))
print("F1 score CNN(from scratch)", round(metrics.f1_score(y_test, pred_val_Y, average='weighted'), 3))

Precision CNN(from scratch) 0.302
Recall CNN(from scratch) 0.35
F1 score CNN(from scratch) 0.298


/usr/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Structure 2
The structure of this convnet is:

Conv2D (input layer) -> Conv2D -> MaxPooling2D -> Conv2D -> Conv2D -> MaxPooling2D -> Dense (relu)-> Conv2D -> Conv2D -> MaxPooling2D -> -> Conv2D -> Conv2D -> MaxPooling2D -> Dense (relu)-> Dense (relu) -> Dense(output, sigmoid, 7)

After our first try, we began to explore some other structured models. VGG-like models are one of our considerations. The inpute shape is 154x154x3. There are 10000 training samples, 5000 validation samples, and 5000 test samples. 


As for the input layer, we set a 5x5 conv window with stride 1. We set 3x3 window with stride 1 for other conv layer. We have the reLU activation function for each conv layer. And we have the factor that used to downscaling to be 2x2 for maxpooling2D layer.

In [14]:
model2 = Sequential()
model2.add(Conv2D(32, (5, 5), activation='relu', input_shape=input_shape))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))


model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(7,activation='sigmoid'))

# prints out a summary of the model architecture
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 70, 70, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 32)        0         
__________

In [15]:
sgd = SGD(lr=0.1, momentum=0.9)
model2.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
history = model2.fit(train_X, train_Y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(vali_X, vali_Y))

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 58s - loss: 3.2632 - acc: 0.7458 - val_loss: 3.6192 - val_acc: 0.7751

In [16]:
score = model2.evaluate(x_test, y_test, verbose=0)

In [17]:
score[1]

0.76628572473526002

In [20]:
#warnings.filterwarnings('ignore')
# F1 score
pred_val_Y= model2.predict(x_test)
print("Precision CNN(from scratch)", round(metrics.precision_score(y_test, pred_val_Y, average='weighted'), 3))
print("Recall CNN(from scratch)", round(metrics.recall_score(y_test, pred_val_Y, average='weighted'), 3))
print("F1 score CNN(from scratch)", round(metrics.f1_score(y_test, pred_val_Y, average='weighted'), 3))

Precision CNN(from scratch) 0.259
Recall CNN(from scratch) 0.35
F1 score CNN(from scratch) 0.297


# Summary for different structure

|          | Structure 1 | Structure 2 |
|----------|-------------|-------------|
| Accuracy | 0.7663      | 0.7661      |
| F1 score | 0.298       | 0.297       |

As we can see from the table above, after adding more convolutional layer and pooling layers, the prediction ability seems not have any improvement.

# 2. Tuning the Dropout rate
In this part we would try different dropout rate to see the influence on the prediction ability.

## Dropout rate (0.1, 0.1, 0.5, 0.5, 0.5)
In this part, we are holding that the structure of the CNN is the same with the model 2.
The structure of this convnet is:

Conv2D (input layer) -> Conv2D -> MaxPooling2D -> Conv2D -> Conv2D -> MaxPooling2D -> Dense (relu)-> Conv2D -> Conv2D -> MaxPooling2D -> -> Conv2D -> Conv2D -> MaxPooling2D -> Dense (relu)-> Dense (relu) -> Dense(output, sigmoid, 7)



In [21]:
model3 = Sequential()
model3.add(Conv2D(32, (5, 5), activation='relu', input_shape=input_shape))
model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.1))

model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.1))

model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.5))

model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.5))


model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(7,activation='sigmoid'))

# prints out a summary of the model architecture
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 150, 32)      2432      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 70, 70, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 35, 35, 32)        0         
__________

In [22]:
sgd = SGD(lr=0.1, momentum=0.9)
model3.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
history = model3.fit(train_X, train_Y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(vali_X, vali_Y))

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 50s - loss: 4.5772 - acc: 0.7047 - val_loss: 4.5648 - val_acc: 0.7153

In [23]:
score = model3.evaluate(x_test, y_test, verbose=0)
score[1]

0.71182858476638788

In [24]:
#warnings.filterwarnings('ignore')
# F1 score
pred_val_Y= model3.predict(x_test)
print("Precision CNN(from scratch)", round(metrics.precision_score(y_test, pred_val_Y, average='weighted'), 3))
print("Recall CNN(from scratch)", round(metrics.recall_score(y_test, pred_val_Y, average='weighted'), 3))
print("F1 score CNN(from scratch)", round(metrics.f1_score(y_test, pred_val_Y, average='weighted'), 3))

Precision CNN(from scratch) 0.304
Recall CNN(from scratch) 0.496
F1 score CNN(from scratch) 0.367


# Summary for different dropout rate

|          | Dropout rate (0.25,0.25,0.25,0.25,0.5) | Dropout rate (0.1,0.1,0.5,0.5,0.5) |
|----------|----------------------------------------|------------------------------------|
| Accuracy | 0.766                                  | 0.712                              |
| F1 score | 0.297                                  | 0.367                              |

As we can see from the table above, different combination may lead to different results. We found that 0.1 and 0.5 combination outperformed the 0.25 and 0.5 combination because we have a better f1 score.

# 3. Tuning the learning rate

In this part we will explore the relationship between learning rate and the prediction ability. In this part we are using the model2. We will try different learning rate and record the acc.

## Learning rate = 10^-5, 0.01, 0.1, 1

In [26]:
lr=[10**-5, 0.01, 0.1, 1]
for r in lr:
    sgd = SGD(lr=r, momentum=0.9)
    model2.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    history = model2.fit(train_X, train_Y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(vali_X, vali_Y))
    score = model2.evaluate(x_test, y_test, verbose=0)
    print('Accuracy for learning rate ',lr,' is ', score[1])
    pred_val_Y=model2.predict(x_test)
    print('F1: ',round(metrics.f1_score(y_test, pred_val_Y, average='weighted'), 3))
    

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 49s - loss: 3.6747 - acc: 0.7716 - val_loss: 3.6192 - val_acc: 0.7751

# Summary for different learning rate

|        lr|      10^-5 | 0.01  | 0.1   | 1     |
|----------|------------|-------|-------|-------|
| Accuracy | 0.766      | 0.766 | 0.766 | 0.766 |
| F1score  | 0.297      | 0.297 | 0.297 | 0.297 |

As we can see from the table above, there is no difference in tuning the learning rate for the SGD optimization process. 

# 4. Tuning the batch size
In this part, we are holding the structure of CNN the same. We used the model2 to fit the data with different batch size.

## Batch size 100, 1000


In [28]:
BS= [200, 100]
for b in BS:
    sgd = SGD(lr=0.1, momentum=0.9)
    model2.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    history = model2.fit(train_X, train_Y,
                        batch_size=b,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(vali_X, vali_Y))
    score = model2.evaluate(x_test, y_test, verbose=0)
    print('Accuracy for batch size ',b,' is ', score[1])
    pred_val_Y=model2.predict(x_test)
    print('F1: ',round(metrics.f1_score(y_test, pred_val_Y, average='weighted'), 3))
    

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 53s - loss: 3.6742 - acc: 0.7717 - val_loss: 3.6192 - val_acc: 0.7751

# Summary for different batch size

|          | batch size =100 | batch size = 200 | batch size = 500 |
|----------|-----------------|------------------|------------------|
| Accuracy | 0.766           | 0.766            | 0.766            |
| F1 score | 0.297           | 0.297            | 0.297            |

As we can see from the above table, there is no significant improvement in prediction ability when we tuned the batch size for SGD optimization process.

# Pre-tuned model - InceptionV3

In [52]:
model4 = InceptionV3(weights=None, include_top=False)
x = model4.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(7, activation='sigmoid')(x)
model_tune = Model(inputs=model4.input, outputs=predictions)

In [53]:
for layer in model_tune.layers:
    layer.trainable = False
model_tune.compile(optimizer='rmsprop', loss='binary_crossentropy',  metrics=['accuracy'])

In [54]:
tune_history = model_tune.fit(train_X, train_Y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(vali_X, vali_Y))

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 55s - loss: 0.6797 - acc: 0.5854 - val_loss: 0.6895 - val_acc: 0.6142

In [55]:
score = model_tune.evaluate(x_test, y_test, verbose=0)
print('Accuracy: ', score[1])

Accuracy:  0.609057167053


In [56]:
pred_val_Y=model_tune.predict(x_test)

In [64]:
Y=np.zeros(pred_val_Y.shape)
for i in range(5000):
    for j in range(7):
        if pred_val_Y[i][j]>0.5:
            Y[i][j]=1
Y

array([[ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.]])

In [65]:
print('F1: ',round(metrics.f1_score(y_test, Y, average='weighted'), 3))

F1:  0.395


# Decision Process
Our idea of the poster genre recognization problem is object detection problem. The key idea in this kind of problem is edge detection and learning. As for this kind of problem, we always believe convolutional neural network works very well. The reason is that convnets usually has less parameters because most of the parameters are shared. In addition, the image usually has lots of redundant information. "Convolution" could help to extract useful information (i.e. edge information). So we decide to go with ConvNets. There are four parts in our convilutional nerual network.
1. **Convolution**  
The concolution step is a very important step at the very beginning. The basic idea is that the convolutional filter could go through the pixel matrix with appropriate stride and extract important features. This convolution step used a small filter/ window  to preserve the spatial relationship among pixels. This could be reasonable becasue most of the pixel data is correlated with each other. 
2. **ReLU**  
The ReLU is an operation that could replace every negative element in the matrix with 0. The reason for the ReLU operation is we want to introduce the non-linearity into our model. The previous convolution step is actually a linear step but in real world, most of the data is non-linear.
3. **Pooling**  
The purpose of the pooling step is that it could help significantly reduce the spatial dimentionality and preserve the feature. Also it could prevent the overfitting by reduce the number of parameters in the model. In addition, it could generate an invariate image representation.
4. **Multi-label classification**  
The fully connected layer which refers to the output layer is equipped with the multi units in it. Each output unit will be embeded a sigmoid operation for each class. The last layer is to do the classification.


# Summary

We divided the dataset into three parts. We have 10000 training samples and 5000 validation samples and 5000 test samples. As for the by scratch model, we have tried different tuning process. We tuned the dropout rate, the learning rate and batch size of he stochastic gradient descent process and we also tried different structured CNN. As for the pre-tuned model, we implemented InceptionV3 model.

The performance of each model is listed below.

|          | Best by scratch model | Pre-tuned model (InceptionV3) |
|----------|-----------------------|-------------------------------|
| Accuracy | 0.712                 | 0.609                         |
| F1 score | 0.367                 | 0.395                         |

As we can see from the table above, both of the best model that we built from scratch and pre-tuned model performed very similar. The strict accuracy for by scratch model is slightly larger than the pre-tuned model but pre-tuned model has larger f1 score. In our situation, we believe that the f1 score makes more sense. The reason is that if the model predicts all 0 or all 1 when the class ratio is imbalanced, the accuracy could be still high. What we really care is that among all positive samples how many are correctly predicted and among all samples predicted to be positive, how many are real positive samples. These two statistics refers to the precision and recall. F1 score is the harmonic mean of precision and recall, so it would be appropriate to compare the model's ability through f1 score. And we can conclude that pre-tuned model performed slightly better.


# Major Challenge

The major challenge we have encountered is that the f1 score for each model is not very high. Also, the most serious problem we have encountered is that the prediction results for each sample is mostly the same. This may indicate the poor ability for the model to capture the feature of each poster and make classification predictions. After re-examining our model and tuning process and adjusting for the imbalance, there seems to be no significant improvement. So we believe the data plays a very important role on the poor prediction ability. We then turned into the data side and went through each poster to see what is actually going on. We found that under each genre, the poster or the data we have has too much noise in it, which means the data or the poster for each genre does not have significant feature to capture or to recognize. Due to these noise in our dataset, the poor prediction ability could be explained. 

# Another idea & Future work

1. Based on the major challenge we have, we came up with a possible solution. Since the major problem in our case is the noise in the data, one possible solution is to denoise the data. What we can think about is to do the denoising manually. That is, we manually pick up some representative posters under each genre. For example, as for the genre crime, we will pick up some posters having guns, blood, and weapons. The problem here in our model is that we failed to capture the feature of each genre. After do the cherry picks, we can guarantee the at least there are some typical features our model could capture in order to have better prediction ability.

2. In this network setting, the response variable is the 7 cluster results. Each column of these 7 columns are binary column, indicating whether this movie is in this cluster of genres or not. The 7 clusters are not mutually exclusive. That is, a movie could be in cluster 1 and 7 at the same time. So the classification in this setting is the multi-label classification. Another idea in the modeling step is to convert the multi-label classification to multi-class classificaation problem. The basic idea is that each cluster represents some genres and each movie have some genres, we then calculate the proportion of this genre in each cluster and we decide the label of this movie with the cluster has the largest proportion. Then we transfer the multi-label problem to the multi-class problem. Then the output layer should only have one unit rather than 7 unit. And instead of the sigmoid unit. We choosed the softmax activation. 

